In [39]:
my $version = "0.12.9";
my @css = grep { length } split /\s+/, qq(
    http://cdn.pydata.org/bokeh/release/bokeh-$version.min.css
    http://cdn.pydata.org/bokeh/release/bokeh-widgets-$version.min.css
    http://cdn.pydata.org/bokeh/release/bokeh-tables-$version.min.css
);
my @js = grep { length } split /\s+/, qq(
    http://cdn.pydata.org/bokeh/release/bokeh-$version.min.js
    http://cdn.pydata.org/bokeh/release/bokeh-widgets-$version.min.js
    http://cdn.pydata.org/bokeh/release/bokeh-tables-$version.min.js
    http://cdn.pydata.org/bokeh/release/bokeh-api-$version.min.js
);


IPerl->display( IPerl->js(<<"JS" )) for @css;
  \$("head").append(\$("<link/>").attr({
    rel: "stylesheet",
    type: "text/css",
    href: "$_"
  }));
JS


IPerl->display( IPerl->js( <<"JS" ) ) for @js;
  \$("head").append(\$("<script>").attr({
    type: "text/javascript",
    src: "$_"
  }));
JS

In [60]:
use PDL;
use feature qw(say);
my $N = 128;

my $xx = zeros($N, $N)->xlinvals(0, 10);
my $yy = zeros($N, $N)->ylinvals(0, 10);
my $d = sin($xx) * cos($yy);

say "Created data"; undef;
my $dtype = $d->type eq 'double' ? 'float64' : undef;

Created data


float64

In [81]:
use JSON::MaybeXS;
use MIME::Base64;
my $json = JSON::MaybeXS->new( allow_nonref => 1 );
my $bkplot_id = "bkplot_001";

my $bokeh_spec = qq|

var source = new Bokeh.ColumnDataSource({
    data: {
        image: {
            shape: [ $N, $N ],
            dtype: "$dtype",
            "__ndarray__": @{[ $json->encode( encode_base64( ${ $d->get_dataref } ) ) ]}
        }
    }
});

var xdr = new Bokeh.Range1d({ start: 0, end: 10 });
var ydr = new Bokeh.Range1d({ start: 0, end: 10 });

var plot = new Bokeh.Plot({
    title: "Bokeh",
    x_range: xdr,
    y_range: ydr
});

var image = new Bokeh.Image({
    image: { field: "image" },
    x: 0,
    y: 0,
    dw: 10,
    dh: 10
    /*,
    color_mapper: "Spectral11"*/
});

//plot.add_glyph(image, source);

// add the plot to a document and display it
var doc = new Bokeh.Document();
doc.add_root(plot);
var div = document.getElementById("$bkplot_id");
Bokeh.embed.add_document_standalone(doc, div);

|;

IPerl->display( IPerl->html( qq|<div id="$bkplot_id" /> | ) );
IPerl->display( IPerl->js( $bokeh_spec ) );